In [1]:
import pandas as pd

import gc
import sys
import os

src_path = os.path.abspath(r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\Credit-Card\src')
sys.path.append(src_path)

from data.clean_data import drop_columns_if_exist, update_dtypes
from data.db_functions import save_to_sqlite

gc.collect()

0

In [2]:
df = pd.read_csv(r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\Credit-Card\data\raw\New_Query_2024-09-07_10_51pm_2024_09_21.csv')
df = df.dropna(axis=1, how='all')

C:\Users\User\AppData\Local\Temp\ipykernel_23812\1197734430.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\Credit-Card\data\raw\New_Query_2024-09-07_10_51pm_2024_09_21.csv')


In [11]:
# Replace non-finite values (NaN, inf) with a placeholder (e.g., -1)
df['card_expiration_month'] = pd.to_numeric(df['card_expiration_month'], errors='coerce').fillna(0).astype(int)
df['card_expiration_year'] = pd.to_numeric(df['card_expiration_year'], errors='coerce').fillna(0).astype(int)


print(f"df shape: {df.shape}")

df shape: (3553562, 28)


In [12]:
# Define database file paths
save_dir = r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\Credit-Card\data\database (SQLite)' 

db_df = os.path.join(save_dir, 'raw_data.db')

In [14]:
save_to_sqlite(df, db_df, 'raw_data')

# Clean up memory
gc.collect()

print("DataFrame has been stored as SQLite databases.")

DataFrame has been stored as SQLite databases.
